In [5]:
! pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [7]:
! pip install static_ffmpeg

  Using cached static_ffmpeg-2.3-py2.py3-none-any.whl (6.5 kB)


In [8]:
! pip install pyaudio

In [30]:
# since the model take only wav file as input , so we'll convert the audio format to wav using pydub

# from math import round
from pydub import AudioSegment
from pydub.playback import play
import static_ffmpeg

static_ffmpeg.add_paths()


audio = AudioSegment.from_file(r"file path here")

# if you want to convert the audio as whole uncomment the code below
# audio.export("D:/reactproj/BE-project/audio/wav_audio.wav", format="wav")

# we'll split the audio in 3 minutes each to minimize the computational power used by model
audio_length = len(audio) / (60 * 1000)

split_marker = 180
split_audio = [audio[:180 * 1000]]

for i in range(round(audio_length / (180 * 1000))):
    split_audio.append(audio[split_marker * 1000:(split_marker + 180) * 1000])
    split_marker += 180

#it will create the file in audio dir, make sure to create audio dir in root folder

count = 0
for count, audio_sample in enumerate(split_audio):
    count += 1
    with open(f"audio/{count}_audi_file.wav", 'wb') as out_f:
        audio_sample.export(out_f, format='wav')

In [20]:
! pip install librosa
! pip install torch
! pip install transformers

  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23694 sha256=9b3a5c70d82c5e4f9be282768db9cbdda5c37b6003f8e625d933f67a646af23b
  Stored in directory: c:\users\saurav singh rauthan\appdata\local\pip\cache\wheels\0a\ed\be\49df2538fca496690a024a4374455584d65c2afd6fc3d6e9c7
Successfully built audioread


In [31]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import os

# loading model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

print("loaded",audio_length)

text_arr = []
for i in range(len(split_audio)):
    # loading audio in model from the audio dir
    speech, rate = librosa.load(f"audio/{i+1}_audi_file.wav", sr=16000) 
    
    input_values = tokenizer(speech, return_tensors='pt').input_values
    with torch.no_grad():
        logits = model(input_values).logits
        
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    text_arr.append(transcription)
    
final_speech = ""
for speech in text_arr:
    final_speech += speech
print(final_speech)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loaded 0.3562666666666667
True
O WHEN I WAS LYING THERE IN THE VIAY HOSPITAL WITH A BIG HALL BLOWN TO THE MIDDLE OF MY LIFE S THOUGH DID HAVING THESE DREAMS OF FLY OS STREE
